In [10]:
import pandas as pd
import numpy as np
# interaction = pd.read_csv("../../data/raw/RAW_interactions.csv")
# pivot = interaction.pivot(index = 'user_id',columns='recipe_id', values='rating').fillna(0)
# pivot.to_csv('data/cleaned/interactions.csv', index = False)

In [11]:
import sys, os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0, parentdir)
 
import src.model as model

In [12]:
interactions = pd.read_csv("../../data/raw/RAW_interactions.csv")

In [13]:
interactions = interactions[["user_id", "recipe_id", "rating"]]
ratinglist = pd.DataFrame(interactions.groupby('user_id')[['recipe_id','rating']]. \
             apply(lambda x: sorted(list(x.values.tolist()), key=lambda y:y[1], reverse = True)), columns=['list'])
ratinglist['count'] = ratinglist.apply(lambda x: len(x['list']), axis = 1)
ratinglist = ratinglist[ratinglist['count'] >= 11].sort_values('count', ascending = False)

In [113]:
ratinglist[-2:].head(3).values

array([[list([[134775, 5], [163419, 5], [54702, 5], [69407, 5], [85847, 5], [54699, 5], [63782, 5], [32522, 4], [68232, 4], [213853, 4], [209794, 4]]),
        11],
       [list([[33919, 5], [86125, 5], [80964, 0], [273998, 0], [133397, 0], [189389, 0], [242922, 0], [84289, 0], [188327, 0], [34110, 0], [299148, 0]]),
        11]], dtype=object)

In [15]:
import src.model as model
all_data = pd.read_csv("../../model/kmeans.csv")

In [16]:
def get_info(row_list, all_data, idx_col = 'id'):
    clust_index_list = [item[0] for item in row_list]
    rating = [item[1] for item in row_list]
    rank = [sorted(rating, reverse = True).index(x)+1 for x in rating]
    info_df = pd.DataFrame(zip(clust_index_list, rating, rank), columns = ['id', 'rating', 'rank']) 
    cluster_data = all_data[all_data[idx_col].isin(clust_index_list)]
    recipe_id = cluster_data[idx_col].iloc[0]
    return recipe_id, cluster_data, info_df

In [17]:
def get_dcg(row):
    return row['rating']/np.log2(row['rank'] if row['rank']>2 else 2)
    
def get_idcg(eval_data, top_n):
    ratelist = eval_data.sort_values('rating', ascending = False).head(top_n)
    idcglist = ratelist.apply(lambda x: get_dcg(x), axis = 1)
    idcg = sum(idcglist)
    return idcg

In [23]:
top_n = 10

result = ratinglist['list'].apply(lambda x: get_info(x, all_data))
ndcg_list = []
debug = []

import logging.config

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.WARNING)

for recipe_id, clust_data, info_df in result:
    if len(clust_data) > 10:
        idxlist, namelist = model.recommend(clust_data, idx=recipe_id)

        # evaluate dataset
        eval_data = info_df[info_df['id'] != recipe_id]
        # idcg
        idcg = get_idcg(eval_data, top_n)
        # dcg
        resultlist = eval_data[eval_data['id'].isin(idxlist)]
        dcglist = resultlist.apply(lambda x: get_dcg(x), axis = 1)
        dcg = sum(dcglist)
        if idcg == 0:
            ndcg = 1
        else:
            ndcg = dcg/idcg
        ndcg_list.append(ndcg)

In [24]:
sum(ndcg_list)/len(ndcg_list)

0.7339816503840267